In [1]:
from transformers import (
    AutoModelForCausalLM,
    AutoTokenizer,
)

model="meta-llama/Meta-Llama-3-8B"
tokenizer = AutoTokenizer.from_pretrained(model)
model = AutoModelForCausalLM.from_pretrained(
    model,
    load_in_4bit=True,
    device_map="auto",
)

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

## 言語モデルの確率の計算方法


In [31]:
tokenizer.pad_token = tokenizer.eos_token

In [46]:
prompt="what language do you like?"
ans = "I like Python"


inputs = tokenizer(prompt,return_tensors="pt",padding=True)
labels = tokenizer(ans,return_tensors="pt",padding=True)

In [48]:
inputs

{'input_ids': tensor([[128000,  12840,   4221,    656,    499,   1093,     30]],
       device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

device(type='cuda', index=0)

In [49]:
import torch

with torch.no_grad():
    logits = model(inputs['input_ids'])

logits=logits.loss['logits']

In [51]:
inputs['input_ids'].shape

torch.Size([1, 7])

In [36]:
labels.shape

torch.Size([1, 4])

In [37]:
logits.shape

torch.Size([1, 7, 128256])

In [38]:
labels[:, 1:]

tensor([[   40,  1093, 13325]])

In [39]:
logits.shape[:-1] == labels.shape

False

In [82]:
label_id

tensor([[   40,  1093, 13325]])

In [84]:
logits.shape

torch.Size([1, 2, 128256])

In [80]:
model.device

device(type='cuda', index=0)

In [81]:
per_token_logps = torch.gather(logits.log_softmax(-1), dim=2, index=label_id.to(model.device).unsqueeze(2)).squeeze(2)


RuntimeError: Size does not match at dimension 1 expected index [1, 3, 1] to be smaller than self [1, 2, 128256] apart from dimension 2

In [74]:
logits.log_softmax(-1)

tensor([[[ -9.2109,  -7.2656,  -3.4883,  ..., -17.5938, -17.5938, -17.5938],
         [ -9.0547, -12.0078, -12.5156,  ..., -22.1094, -22.1094, -22.1094]]],
       device='cuda:0', dtype=torch.float16)

In [28]:
import numpy as np
np.log(probabilities)

tensor(-4.8129)

### データセット

In [ ]:
from datasets import load_dataset

ds = load_dataset("HumanLLMs/Human-Like-DPO-Dataset")

In [ ]:
ds = load_dataset("HumanLLMs/Human-Like-DPO-Dataset")
dataset=ds['train'].shuffle(seed=42).select(range(100))
